<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=de8e5cb05638b6152e22a6e4c5c16049e40e88e2fbf0e9a4aeebea4b851b3d9b
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.1


    Uninstalling protobuf-6.33.1:
      Successfully uninstalled protobuf-6.33.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-11-24 14:28:21
-------------------
qualified stocks: 87
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.26 C
CY Investment:  1.53 C
Reserve:  400.00
Current:  1.42 C
-------------------
Today PnL: -76.97 K (-0.54%)
Current PnL: -23.39 L (-15.33%)
CY Booked + Current PnL: -9.50 L (-6.23%)
-------------------
Total profit:  1.47 L
Total loss:  -24.86 L
-------------------
Total Booked + Current PnL: 17.48 L (13.91%)
Total Booked PnL: 40.87 L (32.53%)
Curr Year Booked PnL: 13.89 L (9.75%)
Prev Year Booked PnL: 26.98 L (21.48%)
Est FTT:  2.31 C
Est FTT PnL: 88.80 L (62.36%)
Deployed:  1.26 C
Current:  1.42 C
CAGR/XIRR %: 7.14%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,TTKPRESTIG,770.00,0.03,-14.21,16.67,0.09,14413.0,-14317.0,86460.0,100.64,49.0,M-SC,4.25,253.0,-0.99,0.61,13.59,OX40N,NTT,DURABLES
39,INDIGOPNTS,1408.00,-0.73,-9.66,10.71,0.02,16890.0,-16856.0,157703.0,140.51,69.0,M-SC,5.40,234.0,-1.00,1.11,37.17,OX40N,NTT,PAINTS
43,ITC,452.00,-0.74,-1.08,11.65,10.44,23064.0,-2166.0,197972.0,-40.74,45.0,X-LC,0.77,5.0,-0.09,1.39,4.42,X40,NTT,FMCG
83,VOLTAS,1530.00,-1.86,7.17,11.69,19.69,24021.0,13743.0,205485.0,-3.29,49.0,X-MC,3.53,78.0,0.57,1.44,14.45,XY25,NTT,AC
35,ICICIGI,2252.93,-0.55,6.33,13.04,20.19,24170.0,11029.0,185349.0,-16.73,49.0,X-MC,7.62,68.0,0.46,1.30,22.40,X40,ATH,INSURANCE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,QUESS,424.00,3.12,-27.10,95.93,42.84,45463.0,-17614.0,47392.0,-52.07,42.0,X-SC,37.19,83.0,-0.39,0.33,3.12,XY24,NTT,MISC
34,HONAUT,58357.33,2.24,-14.29,67.21,43.32,93825.0,-23272.0,139600.0,-26.34,38.0,X-SC,1.95,90.0,-0.25,0.98,7.19,X40N,ATH,ELECTRICAL
14,CAMPUS,393.00,1.86,-9.87,47.88,33.28,72400.0,-16563.0,151212.0,-27.43,42.0,M-SC,2.93,222.0,-0.23,1.06,17.20,XY24,NTT,FOOTWEAR
26,GILLETTE,11206.78,1.59,-0.65,33.29,32.42,83971.0,-1656.0,252240.0,-18.36,36.0,X-SC,6.64,85.0,-0.02,1.77,14.35,X40,ATH,FMCG
29,HAPPSTMNDS,1480.71,1.58,-36.99,193.36,84.85,156158.0,-47405.0,80760.0,-25.05,47.0,H-SC,14.26,147.0,-0.30,0.57,3.65,AR,ATH,IT


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
81,VALIANTORG,838.00,-3.95,-46.38,224.81,74.15,197203.0,-75885.0,87720.0,-257.83,25.0,H-SC,14.60,149.0,-0.38,0.62,12.44,XR,NTT,CHEMICALS
58,RAJOOENG,143.10,-3.62,-30.19,99.97,39.61,71539.0,-30940.0,71560.0,-52.76,20.0,H-SC,19.29,135.0,-0.43,0.50,0.00,AR,ATH,MISC
65,SHALBY,327.00,-3.22,-19.58,55.56,25.10,80938.0,-35467.0,145676.0,971.77,27.0,M-SC,16.44,250.0,-0.44,1.02,16.74,XY24,NTT,HEALTHCARE
27,GLAXO,3466.20,-3.19,-2.93,43.34,39.15,83840.0,-5836.0,193448.0,-22.01,27.0,X-MC,9.10,60.0,-0.07,1.36,25.22,X40,ATH,PHARMA
72,TANLA,1943.92,-3.13,-36.77,247.44,119.69,423635.0,-99554.0,171207.0,-36.34,26.0,H-SC,12.01,156.0,-0.23,1.20,35.25,AR,ATH,IT


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
38,INDIAMART,4810.62,-1.4,-0.78,108.34,106.72,132583.0,-959.0,122377.0,-50.82,32.0,H-SC,4.06,139.0,-0.01,0.86,22.42,AR,ATH,MISC


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,-1.42,-5.34,119.65,107.91,167216.0,-7890.0,139754.0,-22.98,32.0,M-SC,10.37,216.0,-0.05,0.98,0.00,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,-0.73,-9.66,10.71,0.02,16890.0,-16856.0,157703.0,140.51,69.0,M-SC,5.40,234.0,-1.00,1.11,37.17,OX40N,NTT,PAINTS
78,TTKPRESTIG,770.00,0.03,-14.21,16.67,0.09,14413.0,-14317.0,86460.0,100.64,49.0,M-SC,4.25,253.0,-0.99,0.61,13.59,OX40N,NTT,DURABLES
47,KANSAINER,340.00,-0.38,-22.60,46.61,13.47,97284.0,-60948.0,208719.0,-68.12,26.0,H-SC,1.52,158.0,-0.63,1.47,6.13,XY24,NTT,PAINTS
73,TATAELXSI,9161.00,0.27,-23.11,71.72,32.03,72699.0,-30470.0,101365.0,-16.88,48.0,H-SC,6.64,148.0,-0.42,0.71,13.54,OX40N,NTT,IT


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,WIPRO,420.0,1.08,1.36,69.96,72.27,120155.0,2300.0,171748.0,-12.89,57.0,M-LC,4.62,99.0,0.02,1.21,7.62,XR,NTT,IT


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,WIPRO,420.00,1.08,1.36,69.96,72.27,120155.0,2300.0,171748.0,-12.89,57.0,M-LC,4.62,99.0,0.02,1.21,7.62,XR,NTT,IT
38,INDIAMART,4810.62,-1.40,-0.78,108.34,106.72,132583.0,-959.0,122377.0,-50.82,32.0,H-SC,4.06,139.0,-0.01,0.86,22.42,AR,ATH,MISC
25,FINCABLES,1641.55,-1.42,-5.34,119.65,107.91,167216.0,-7890.0,139754.0,-22.98,32.0,M-SC,10.37,216.0,-0.05,0.98,0.00,OX40N,ATH,CABLES
84,WHIRLPOOL,2270.00,-0.27,-5.67,92.85,81.92,100550.0,-6505.0,108293.0,-44.81,31.0,M-SC,5.31,236.0,-0.06,0.76,28.20,XR,NTT,DURABLES
86,ZYDUSLIFE,1286.17,-0.06,-2.59,39.24,35.63,80104.0,-5436.0,204138.0,-16.30,32.0,H-MC,4.51,119.0,-0.07,1.43,13.65,AR,ATH,PHARMA


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
46,JSWINFRA,342.0,-1.02,-9.99,28.02,15.23,50527.0,-20021.0,180326.0,-22.98,23.0,X-MC,6.51,66.0,-0.40,1.27,19.03,X40N,NTT,REALTY
76,TMPV,600.0,-0.81,-14.86,66.99,42.18,157656.0,-41068.0,235342.0,-23.08,23.0,X-LC,0.40,3.0,-0.26,1.65,0.00,XY24,NTT,AUTO
27,GLAXO,3466.2,-3.19,-2.93,43.34,39.15,83840.0,-5836.0,193448.0,-22.01,27.0,X-MC,9.10,60.0,-0.07,1.36,25.22,X40,ATH,PHARMA
11,BATAINDIA,2096.0,0.82,-34.94,107.81,35.20,90252.0,-44956.0,83714.0,9.75,27.0,X-SC,12.60,92.0,-0.50,0.59,0.82,X40,NTT,FOOTWEAR
60,RELAXO,1176.0,-0.94,-47.60,190.66,52.31,145026.0,-69095.0,76065.0,-44.25,28.0,X-SC,3.79,91.0,-0.48,0.53,1.82,X40N,NTT,FOOTWEAR


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
76,TMPV,600.0,-0.81,-14.86,66.99,42.18,157656.0,-41068.0,235342.0,-23.08,23.0,X-LC,0.40,3.0,-0.26,1.65,0.00,XY24,NTT,AUTO
43,ITC,452.0,-0.74,-1.08,11.65,10.44,23064.0,-2166.0,197972.0,-40.74,45.0,X-LC,0.77,5.0,-0.09,1.39,4.42,X40,NTT,FMCG
36,ICICIPRULI,790.0,0.30,1.58,28.93,30.97,51940.0,2798.0,179536.0,-22.29,50.0,X-MC,1.54,75.0,0.05,1.26,14.34,X40,ATH,INSURANCE
3,ACC,3906.0,-0.89,-23.77,115.34,64.15,209215.0,-56561.0,181390.0,-55.44,39.0,X-SC,1.92,82.0,-0.27,1.27,1.75,XY24,BTT,CEMENT
66,SIEMENS,4671.5,0.18,-14.58,46.94,25.51,74616.0,-27135.0,158960.0,-2.45,55.0,H-LC,1.93,49.0,-0.36,1.12,17.14,AR,ATH,ELECTRICAL


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
76,TMPV,600.00,-0.81,-14.86,66.99,42.18,157656.0,-41068.0,235342.0,-23.08,23.0,X-LC,0.40,3.0,-0.26,1.65,0.00,XY24,NTT,AUTO
9,BAJAJHFL,181.50,0.22,-13.50,71.61,48.45,124205.0,-27060.0,173446.0,-19.66,31.0,X-MC,7.46,64.0,-0.22,1.22,0.22,X40N,ATH,FINANCE
54,PAGEIND,51605.07,-0.12,-5.12,32.87,26.07,51067.0,-8380.0,155360.0,-30.18,35.0,X-MC,8.87,55.0,-0.16,1.09,0.71,X40,ATH,APPARELS
55,PGHH,17907.65,-0.19,-4.20,39.62,33.76,76225.0,-8430.0,192390.0,-32.38,34.0,X-MC,3.20,57.0,-0.11,1.35,0.78,X40,ATH,FMCG
11,BATAINDIA,2096.00,0.82,-34.94,107.81,35.20,90252.0,-44956.0,83714.0,9.75,27.0,X-SC,12.60,92.0,-0.50,0.59,0.82,X40,NTT,FOOTWEAR


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,QUESS,424.00,3.12,-27.10,95.93,42.84,45463.0,-17614.0,47392.0,-52.07,42.0,X-SC,37.19,83.0,-0.39,0.33,3.12,XY24,NTT,MISC
60,RELAXO,1176.00,-0.94,-47.60,190.66,52.31,145026.0,-69095.0,76065.0,-44.25,28.0,X-SC,3.79,91.0,-0.48,0.53,1.82,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,0.82,-34.94,107.81,35.20,90252.0,-44956.0,83714.0,9.75,27.0,X-SC,12.60,92.0,-0.50,0.59,0.82,X40,NTT,FOOTWEAR
52,MEDANTA,1486.00,0.55,3.00,20.46,24.08,26754.0,3812.0,130762.0,-5.33,45.0,X-SC,4.74,89.0,0.14,0.92,23.21,XY24,NTT,HEALTHCARE
22,DIXON,18931.72,-1.57,-5.40,28.52,21.58,37809.0,-7569.0,132570.0,-52.33,32.0,X-MC,7.04,56.0,-0.20,0.93,17.37,X40N,ATH,IT


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4389.97,0.62,-13.87,38.48,19.28,114663.0,-47976.0,297980.0,-24.92,66.0,X-LC,7.63,1.0,-0.42,2.09,10.16,X40,ATH,IT
41,INFY,2275.00,0.63,5.84,46.33,54.87,153422.0,18263.0,331151.0,-17.44,62.0,X-LC,3.16,2.0,0.12,2.33,14.60,X40,BTT,IT
76,TMPV,600.00,-0.81,-14.86,66.99,42.18,157656.0,-41068.0,235342.0,-23.08,23.0,X-LC,0.40,3.0,-0.26,1.65,0.00,XY24,NTT,AUTO
82,VBL,671.64,0.02,-9.55,50.00,35.67,142832.0,-30178.0,285664.0,-20.30,41.0,X-LC,6.05,4.0,-0.21,2.01,2.98,X40N,ATH,FMCG
43,ITC,452.00,-0.74,-1.08,11.65,10.44,23064.0,-2166.0,197972.0,-40.74,45.0,X-LC,0.77,5.0,-0.09,1.39,4.42,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.00,-0.79,-13.45,109.90,81.67,89234.0,-12614.0,81196.0,7300.00,50.0,L-SC,18.57,271.0,-0.14,0.57,57.24,XR,NTT,CERAMICS
68,SONACOMS,806.63,0.33,-12.67,59.79,39.54,52818.0,-12819.0,88340.0,-31.70,67.0,M-SC,6.82,220.0,-0.24,0.62,24.57,AR,ATH,AUTO
51,MASFIN,398.61,-2.11,-4.98,28.44,22.05,26479.0,-4875.0,93105.0,-18.25,50.0,H-SC,7.21,164.0,-0.18,0.65,35.33,XR,ATH,FINANCE
70,SURYODAY,216.00,0.52,-19.24,52.35,23.03,75706.0,-34455.0,144616.0,57.89,49.0,H-SC,11.14,167.0,-0.46,1.02,43.49,XR,NTT,BANKS
39,INDIGOPNTS,1408.00,-0.73,-9.66,10.71,0.02,16890.0,-16856.0,157703.0,140.51,69.0,M-SC,5.40,234.0,-1.00,1.11,37.17,OX40N,NTT,PAINTS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
39,INDIGOPNTS,1408.00,-0.73,-9.66,10.71,0.02,16890.0,-16856.0,157703.0,140.51,69.0,M-SC,5.40,234.0,-1.00,1.11,37.17,OX40N,NTT,PAINTS
5,ASIANPAINT,3460.25,0.12,0.66,20.15,20.95,51068.0,1672.0,253440.0,-0.35,76.0,X-LC,12.74,31.0,0.03,1.78,36.54,X40,ATH,PAINTS
40,INDUSINDBK,1800.00,-0.42,-37.76,113.52,32.89,54547.0,-29156.0,48051.0,-693.69,62.0,L-MC,5.60,259.0,-0.53,0.34,32.33,XR,NTT,BANKS
50,LTIM,7201.88,-0.05,8.28,21.59,31.66,57545.0,20384.0,266535.0,3.99,62.0,H-LC,12.47,43.0,0.35,1.87,47.89,X200,ATH,IT
0,5PAISA,593.00,-2.83,-36.57,78.00,12.90,108100.0,-79906.0,138590.0,104.33,55.0,H-SC,14.60,161.0,-0.74,0.97,15.24,OX40N,NTT,FINANCE


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.83
1,25,44.70
2,50,75.45


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    45.50
MC    29.25
LC    25.21
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     27.84
X40      21.24
X40N     11.89
XR        9.64
XY25      9.09
AR        8.95
OX40N     7.68
X200      1.87
SR        0.99
MH        0.77
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    23.89
X-MC    22.84
X-LC    20.04
M-SC    12.00
X-SC     8.21
H-MC     4.70
H-LC     2.99
L-SC     1.40
M-MC     1.37
M-LC     1.21
L-LC     0.97
L-MC     0.34
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,16.81,-5.07,39.05
IT,13.85,-14.67,73.87
FINANCE,9.40,-15.74,64.65
MISC,7.39,-23.09,73.52
PAINTS,6.00,-8.12,24.18
ELECTRICAL,5.92,-11.38,52.17
INSURANCE,4.42,-0.42,35.81
PHARMA,3.83,-2.44,35.22
AUTO,3.35,-20.39,72.01


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3068171.0,21
AR,1297323.0,10
XR,1290482.0,13
X40,1013477.0,14
X40N,755567.0,9
OX40N,698169.0,10
XY25,345563.0,6
SR,279931.0,2
MH,74114.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3576724.0,25
M-SC,1348230.0,15
X-MC,1229189.0,16
X-LC,908418.0,11
X-SC,749097.0,8
H-MC,408621.0,3
L-SC,261968.0,3
H-LC,132161.0,2
M-LC,120155.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1225617.0      6
           AR         881204.0      5
           XR         791972.0      7
M-SC       XY24       788090.0      6
X-MC       X40        449199.0      7
X-LC       X40        390055.0      5
X-MC       XY24       350324.0      3
H-SC       OX40N      323886.0      4
X-SC       X40N       293442.0      3
M-SC       OX40N      286220.0      5
X-SC       XY24       281432.0      3
H-SC       SR         279931.0      2
X-MC       X40N       249361.0      4
X-LC       XY24       231573.0      2
H-MC       AR         217486.0      2
X-LC       X40N       212764.0      2
H-MC       XY24       191135.0      1
X-MC       XY25       180305.0      2
X-SC       X40        174223.0      2
L-SC       XR         173905.0      2
M-SC       XR         149903.0      2
           AR         124017.0      2
M-LC       XR         120155.0      1
L-SC       OX40N       88063.0      1
H-LC       AR          74616.0      1
H-SC       MH          74114.0      1
X-LC       XY25        74026.0      2
H-LC       X200        57545.0      1
L-MC       XR          54547.0      1
M-MC       XY25        49857.0      1
L-LC       XY25        41375.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 28.0 seconds
